# Library mocker for unit tests and validation

In [1]:
Mock = require ("mock.mock") -- imports mock and it lib
It = require("mock.it")

#### simple Assert function

In [2]:
function Assert(a, b)

    if a==b then 
    print("test passed ✔")
    else print("test failed ❌")
    end
end 

### Dummy code 

* `HTTP` represent a dependency or library
* `BL` is a dummy business logic that uses the `HTTP` lib
* `downloader` is what we wants to tests

For this demo we need two objects, the first is what we are actually interested in testing, we will call it **SUT** or system under test, the second object is the collaboator.

In [3]:

-- business-logic

local HTTP = require("HTTP")

Fetch = {
    http_client = HTTP.New({}) 
}

function Fetch:GET(url)
    
    return self.http_client:Get(url)
end

function Fetch:POST(url, data)
    return self.http_client:Post(url, data)
end


### SUT (System under test)

`downloader`: function represents the piece of code we want to test, it has two arguments:
* `cache`: is local varibale that stores previously feteched content, so we don't make unnecessary calls.
* `fetch`: is the object will be used to make the http request.
* `url`:  end point.

In [4]:

function downloader(fetch, url) 
   return  fetch:GET(url)

end

## **1.** Dummy Object
we can use the mock object as Dummy object.
Dummy objects are passed around but never actually used. Usually they are just used to fill parameter lists

In [5]:

local mock = Mock:new(Fetch) -- create an instance of the mock object passing the object we wants to mock
downloader(mock, "www.example.com") -- our mock object will be used only to fill gaps 

## **2.** Mock object 
objects pre-programmed with expectations which form a specification of the calls they are expected to receive

#### 2.2 setting function behavior with the `returns` and `with` functions

In [6]:

-- arrange

local expected_result = "some random result"
local fetch_mock = Mock:new(Fetch) 
fetch_mock:setup(Fetch.GET):with("www.example.com"):returns(expected_result)  

-- act

local result = downloader(fetch_mock, "www.example.com")

-- assert

Assert(result, expected_result)


test passed ✔ 


#### 2.3 using type based args

by using the **`It`** class you can pass type only verification without checking the value.
* `It.any_string()` 
* `It.any_number()` 
* `It.any_int()` 
* `It.any_float()` 
* `It.any_table()` 
* `It.pattern()` 

In [7]:
-- arrange

local expected_result = "some result"
local fetch_mock = Mock:new(Fetch) 
fetch_mock:setup(Fetch.GET):with(It.any_string()):returns(expected_result)  

-- act

local result = downloader(fetch_mock, "www.example.com")

-- assert

Assert(result, expected_result)

test passed ✔ 


In [8]:
-- arrange

local expected_result = "some result"
local fetch_mock = Mock:new(Fetch) 
fetch_mock:setup(Fetch.GET):with(It.pattern("https://www%.%w+%.%w+")):returns(expected_result)  -- using pattern base verification

-- act

local result = downloader(fetch_mock, "www.example.com")

-- assert

Assert(result, expected_result)

test passed ✔ 


#### 2.4 Float and Int args verification

In [9]:
-- we will modify the downloader function to accept 3rd parameter id and it should be integer type
function downloader(fetch, url, id) 

   return  fetch:GET(url, id)

end

In [10]:
local expected_result = "some random result"
local fetch_mock = Mock:new(Fetch) 
fetch_mock:setup(Fetch.GET):with( It.any_string(), It.any_int()):returns(expected_result) 

-- act

local result = downloader(fetch_mock,"www.example.com", 1003)

-- assert

Assert(result, expected_result)

test passed ✔ 


In [11]:
local expected_result = "some random result"
local fetch_mock = Mock:new(Fetch) 
fetch_mock:setup(Fetch.GET):with( It.any_string(), It.any_int()):returns(expected_result)  --

-- act

local result = downloader(fetch_mock,"www.example.com", 1003.4) -- Error : passing float, should be integer

-- assert

Assert(result, expected_result)

./mock/func.lua:83: different args was passed to the function: ./mock/func.lua:83: different args was passed to the function

## 3. Strict Mode 

strict mode will track the mock object and any method that has not been set will result in error

In [12]:
-- modify the downloader to call post which is not set yet
function downloader(fetch, url) 
    
    fetch:POST(url, 3) --  calling POST with out setting it
   return  fetch:GET(url)

end

In [13]:

-- arrange
local expected_result = "some random result"
local fetch_mock = Mock:new(Fetch, true) -- stric mode is activated passing true to the second parameter 
fetch_mock:setup(Fetch.GET):with("www.example.com"):returns(expected_result)  

-- act

local result = downloader(fetch_mock, "www.example.com")

-- assert

Assert(result, expected_result)

./mock/mock.lua:28: POST is called but was not set. you can set POST or strict mode to false: ./mock/mock.lua:28: POST is called but was not set. you can set POST or strict mode to false

## 4. Behavior testing

we can test the behavior of SUT by enabling the strict mode and calling the run counter methods

In [18]:
-- not the downloader has a cache table 
-- it should run the GET request only once for each url
local cache  = {}
function downloader(fetch, url) 

    if cache[url] then return cache[url]

    else 
        local result  = fetch:GET(url)
        cache[url] = result
        return result
    end 
end

In [19]:
-- arrange
local expected_result = "some random result"
local fetch_mock = Mock:new(Fetch, true) -- stric mode is activated passing true to the second parameter 
fetch_mock:setup(Fetch.GET):with("www.example.com"):returns(expected_result):run_once()

-- act

local result = downloader(fetch_mock, "www.example.com")

-- assert

Assert(result, expected_result)

test passed ✔ 


In [20]:
-- not the downloader has a cache table 
-- it should run the GET request only once for each url
local cache  = {}
function downloader(fetch, url) 

    if cache[url] then return cache[url]

    else 
        local result  = fetch:GET(url) 
        local result  = fetch:GET(url) --> Error: we call the GET twice
        cache[url] = result
        return result
    end 
end

In [21]:
-- arrange
local expected_result = "some random result"
local fetch_mock = Mock:new(Fetch, true) -- stric mode is activated passing true to the second parameter 
fetch_mock:setup(Fetch.GET):with("www.example.com"):returns(expected_result):run_once()

-- act

local result = downloader(fetch_mock, "www.example.com")

-- assert

Assert(result, expected_result)

./mock/func.lua:70: GET should be called exactly once, but it was called 2: ./mock/func.lua:70: GET should be called exactly once, but it was called 2